In [2]:
# %load apple.py
import requests
from bs4 import BeautifulSoup
import datetime
import re
import MySQLdb
import sys,time


In [ ]:
#choose day
begin = datetime.date(2018,9,21) 
end = datetime.date(2018,9,20)
d = begin
#count day by delta 
delta = datetime.timedelta(days=1)
data= []

In [ ]:
#while util day to end
while d >= end:
    print(d)
    date = d.strftime("%Y%m%d")
    date_url = 'https://tw.appledaily.com/appledaily/archive/' + date #date of apple news
    res = requests.get(date_url)
    soup = BeautifulSoup(res.text,'html.parser')
    articles = soup.select('ul.fillup li a')  #aritles
     
    for article in articles:
        try:
            if article['href'].find("http://") != -1 or article['href'].find("https://") != -1:
            #judge the url is not right
            #if article['href'].find("http://")!=-1:
                 article_url = article['href']
                 res = requests.get(article_url)              
                 soup = BeautifulSoup(res.text,'html.parser')    
                 if article_url.find("home") == -1: #board
                    title = soup.select('hgroup h1')[0]  
                    time = soup.select('div.ndArticle_creat') [0]
                    content = soup.select('.ndArticle_margin p') 
                 else:
                     title = soup.select('.ncbox_cont h1') [0]               
                     content = soup.select('p')
                     time = soup.select('div time')[0]
                 title = re.sub('\[*<+/?\w+[^>]*>+\]*','',str(title))     
                 content = re.sub(r'\[*<+/?\w+[^>]*>+\]*','',str(content))
                 time = re.sub(r'\[*<+/?\w+[^>]*>+\]*','',str(time))
                #assign the board from the url 
                 if article_url.find("headline/daily/") != -1: 
                     board = "headline"
                 elif article_url.find("entertainment") != -1:
                    board = "entertainment"
                 elif article_url.find("international") != -1:
                    board = "international"
                 elif article_url.find("sports") != -1:
                    board = "sports"
                 elif article_url.find("finance") != -1:
                    board = "finance"
                 elif article_url.find("home") != -1:   
                    board ="home"
                 if article_url.find("lifestyle") != -1:
                    board = "lifestyle"
                 elif article_url.find("forum") != -1:
                    board = "forum"
    ##                 dic = {'title' : title , 'time' : time , 'board' : board , 'content':content, 'url':article_url}
    ##                 data.append(dic)
                 db = MySQLdb.connect("database","account", "password", "table")
                 cursor = db.cursor()
                 db.set_character_set('utf8')
                 cursor.execute('SET NAMES utf8;')
                 cursor.execute('SET CHARACTER SET utf8;')
                 cursor.execute('SET character_set_connection=utf8;')
                #insert new apple news into the database
                 insert_apple_news = ("INSERT INTO " +board+"_" +d.strftime("%Y")+"(Title,Time,Board,Content,Url)" "VALUES(%s,%s,%s,%s,%s)")
                #update the older the apple news
                #update_apple_news = ("UPDATE" +board+"_" +d.strftime("%Y")+" SET Content = %s WHERE Title= %s")
                 data_apple_news = (title,d,board,content,article_url)
                 cursor.execute(insert_apple_news, data_apple_news)
                 #cursor.execute("UPDATE " +board+"_" +d.strftime("%Y")+" SET Content = %s WHERE Title= %s" , (content,title))
                 db.commit()
                 #print("INSERT INTO " +board+"_" +d.strftime("%Y")+"(Title,Time,Board,Content,Url)" "VALUES(%s,%s,%s,%s,%s)")
        except:
                print(article_url +  "    error ")
                continue   
               
    d -= delta 
    db.close()

        


In [ ]:
#export to csv
##table = pandas.DataFrame(data)
##table
##table.to_csv('out.csv')
